# Tutorial: Inspect Planner, Worker, and Verifier I/O

Notebook goals:
- Inspect input/output boundaries of `Planner (Architect)`, `Worker`, and `Verifier (Critic)`.
- Compare behavioral differences between `codeact` and `react`.
- Build a foundation for role-specific trace review and dataset tuning.

## Outline
1. Set up imports and a deterministic mock runtime.
2. Inspect Planner I/O.
3. Inspect Worker I/O (`CodeAct` vs `ReAct`).
4. Inspect Verifier I/O under normal and error observations.

In [ ]:
from __future__ import annotations

import json
import sys
from pathlib import Path


def find_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / "pyproject.toml").exists() and (candidate / "src").exists():
            return candidate
    raise RuntimeError("Could not locate repo root")


REPO_ROOT = find_repo_root(Path.cwd())
if str(REPO_ROOT / "src") not in sys.path:
    sys.path.insert(0, str(REPO_ROOT / "src"))

from manus_three_agent.agents import ArchitectAgent, CriticAgent, WorkerAgent
from manus_three_agent.core.types import ModelConfig
from manus_three_agent.prompts import PromptTemplates
from manus_three_agent.tools import build_default_tool_registry
from manus_three_agent.tracing import TraceCollector

print(f"Repo root: {REPO_ROOT}")


## Step 1: Planner (Architect) I/O

In [ ]:
prompts = PromptTemplates(config_dir=str(REPO_ROOT / "configs" / "prompts"))
model_cfg = ModelConfig(model="mock")
tracer = TraceCollector.disabled()

goal = "Design an evaluation plan for LLM agent trajectory quality"
observation = "Environment ready. No previous actions."
action_history: list[dict] = []

architect = ArchitectAgent(
    model_cfg,
    prompts,
    tracer=tracer,
    force_mock=True,
    agentic_mode="codeact",
)

plan = architect.plan(
    goal=goal,
    observation=observation,
    action_history=action_history,
    use_cot=False,
    step=0,
)

planner_io = {
    "input": {
        "goal": goal,
        "observation": observation,
        "action_history": action_history,
    },
    "output": plan.model_dump(),
}
print(json.dumps(planner_io, indent=2, ensure_ascii=False))


**Expected output**
- `output.steps` is a list of plan steps with `title` and `rationale`.
- This is the primary contract consumed by the downstream Worker.

## Step 2: Worker I/O (CodeAct vs ReAct)

In [ ]:
tools = build_default_tool_registry()
plan_step = plan.steps[0]
worker_outputs: dict[str, dict] = {}

for mode in ["codeact", "react"]:
    worker = WorkerAgent(
        model_cfg,
        prompts,
        tools,
        tracer=tracer,
        force_mock=True,
        agentic_mode=mode,
    )
    action = worker.execute(
        goal=goal,
        plan_step=plan_step,
        observation=observation,
        step_index=0,
        total_steps=len(plan.steps),
        use_cot=False,
        step=1,
    )
    worker_outputs[mode] = action.model_dump()

print(json.dumps(worker_outputs, indent=2, ensure_ascii=False))


**Expected output**
- `codeact` generally emphasizes action/tool-grounded behavior.
- `react` generally emphasizes iterative reasoning summaries.

## Step 3: Verifier (Critic) I/O

In [ ]:
verifier_outputs: dict[str, dict] = {}

for mode, obs in [
    ("codeact", "Step done without errors."),
    ("react", "Step failed with malformed tool output."),
]:
    critic = CriticAgent(
        model_cfg,
        prompts,
        tracer=tracer,
        force_mock=True,
        agentic_mode=mode,
    )
    review = critic.review(
        goal=goal,
        observation=obs,
        action_history=[worker_outputs[mode]],
        current_step_idx=1,
        plan_length=len(plan.steps),
        step=1,
    )
    verifier_outputs[f"{mode}:{obs}"] = review.model_dump()

print(json.dumps(verifier_outputs, indent=2, ensure_ascii=False))


## Takeaways
- Planner/Worker/Verifier expose clear and typed I/O contracts.
- `agentic_mode` can alter behavior profiles without changing orchestration core logic.
- These traces provide a strong entry point for role-specific trajectory datasets.